In [ ]:
import pandas as pd
from google.colab import auth
from google.colab import files
auth.authenticate_user()
print('Authenticated')

In [ ]:
city_data = pd.read_csv('processed.csv')
cities = []
subreddits = []
for city in city_data['Metro']:
    city_name = city.split('-')[0]
    city_name = city_name.split(',')[0]
    city_name = city_name.split('/')[0]
    cities.append(city_name)
    subreddits.append(''.join(city_name.lower().split()))

cities[0]='NYC'
cities[45]='Salt Lake'
subreddits[0]='nyc'

subreddits = ['nyc', 'LosAngeles', 'chicago', 'Dallas', 'houston', 'washingtondc', 'Miami', 'philadelphia', 'Atlanta', 'phoenix', 'boston', 'sanfrancisco', 'Riverside', 'Detroit', 'Seattle', 'Minneapolis', 'sandiego', 'tampa', 'Denver', 'StLouis', 'baltimore', 'Charlotte', 'orlando', 'sanantonio', 'Portland', 'Sacramento', 'pittsburgh', 'LasVegas', 'Austin', 'cincinnati', 'kansascity', 'Columbus', 'indianapolis', 'Cleveland', 'SanJose', 'nashville', 'VirginiaBeach', 'providence', 'milwaukee', 'jacksonville', 'oklahomacity', 'raleigh', 'richmond', 'NewOrleans', 'Louisville', 'SaltLakeCity', 'Hartford', 'Buffalo', 'Birmingham']

dates=[]
for i in range(1,13):
    for year in range(5,9):
        month = '0{}'.format(i)[-2:]
        dates.append('201{}_{}'.format(year, month))

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project='cse6741')

for city_idx in range(len(cities)):
    bodies = []
    subrs = []
    authors = []

    for date in dates:
        print(date, cities[city_idx],len(bodies))
        query = """
                SELECT body,subreddit,author 
                FROM `fh-bigquery.reddit_comments.{}` 
                WHERE (body LIKE @string OR body LIKE @string2) AND (subreddit = @subreddit)
                """.format(date)

        job_config = bigquery.QueryJobConfig(
            query_parameters=[bigquery.ScalarQueryParameter("string", "STRING", '%{}%'.format(cities[city_idx])),
                              bigquery.ScalarQueryParameter("string2", "STRING", '%{}%'.format(cities[city_idx].lower())),
                              bigquery.ScalarQueryParameter("subreddit", "STRING", subreddits[city_idx]),])
        query_job = client.query(query, job_config=job_config)  # Make an API request.

        for row in query_job:
            bodies.append(row.body)
            subrs.append(row.subreddit)
            authors.append(row.author)

    df = pd.DataFrame({'body':bodies,'subreddit':subrs,'author':authors})
    df.to_csv('{}.csv'.format(cities[city_idx]),header=1)